<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_6_Sampling_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Introduction**
The goal of this notebook is to generate a set of point locations for our case study where we will validate the performance of each flood product/package based on high-resolution optical imagery. To do this, we need our flood maps from each product/package as well as our high-resolution optical imagery. We will conduct a clustered stratified sampling approach.

## Step 1: Clustering

Our first step is to cluster our area of interest. In this case, we will have one cluster that will consist of areas where **no** satellite has an obstructed view of the ground due to cloud cover, haze, or low quality data.


For our optical flood maps, each of these maps comes with a cloud or low data quality mask. In Module 6, we assigned all of the maps the common value of 2 for this unusable data. We also want to mask out areas where our high-res optical data from Planet has clouds. We can use the Usable Data Mask (UDM) that comes with Planet data for this purpose.

We will find the **union** of areas where each optical sensor has unusable data, and remove this union from the area of analysis.

## Step 2: Stratification

Stratification refers to the splitting of the area of interest into strata, which are groups of pixels that are both exhaustive and exlcusive. In other words, each pixel in the area of interest can be assigned to one and only one strata. In this case, we will use three classes of strata to investigate the performance of the flood products across various types of surface conditions. The three classes of strata we will use are land cover, elevation and slope


### Step 2 Part 1: Land Cover

We will use Google's Dynamic World Dataset [Brown et al 2022] to stratify the sampling area by Land Cover

### Step 2 Part 2: Product Output

We will use the output of the flood products themselves (i.e. water, nonwater) to evaluate the performance of the products by class.

# Step 1: Define input Variables


Change the input variables below to suit your needs

In [ ]:
# Google Earth Engine Directory
parent_directory = "projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/"

# Step 2: Import packages

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt

If you have not already installed geemap, uncomment the cell below.

In [ ]:
#!pip install geemap

Now that we have installed our packages, it's time to import them via import so we can use them in this notebook. Below you can find a brief description of what each package does.

The geemap package will allow us to visualize a Google Maps interface interactively within this notebook.
The ee package will allow us to run Earth Engine functions via the python programming language.

In [ ]:
import geemap
import ee

In order to continue running this script, you need to be associated with a Google Cloud Project. Now, we have to authenticate and initialize earth engine. After you run the code below, click through the pop-up window to login to the Google Account associated with your Google Earth Engine account. Click "Continue" until you have returned to this notebook and a green checkmark appears to the left of the code cell below.

In [ ]:
ee.Authenticate()
ee.Initialize(project = 'servir-sco-assets')

Import the Outputs from the previous modules:



*   The Planet Imagery Usable Data Masks
*   Each Flood Map



In [ ]:
aoi = ee.FeatureCollection(parent_directory + "aoi")
'''
# Planet Imagery
plc1 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/classic_one")           # Planet Classic Part 1
plc2 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/classic_pt2_composite") # Planet Classic Part 2
pldr = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/dove_r_composite")      # Planet Dove R
pls1 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/superdoveone")          # Planet SuperDove 1
pls2 = ee.Image("projects/servir-sco-assets/assets/sd_0530_pt1")                                                  # Planet SuperDove 2

# Planet UDMs
plc1u = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/classic_pt1_composite_udm2")   # Planet Classic Part 1 UDM (Usable Data Mask)
plc2u = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/classic_pt2_composite_udm2")   # Planet Classic Part 2 UDM (Usable Data Mask)
plru = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/dove_r_composite_udm2")         # Planet Dove R UDM
pls1u = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/superdove_pt1_composite_udm2") # Planet Superdove Part 1 UDM (Usable Data Mask)
pls2u = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/pl/superdove_pt2_composite_udm2") # Planet Superdove Part 2 UDM (Usable Data Mask)
'''
# Planet Imagery
pls1p1 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt1/swath1_pt1")
pls2p1 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt1")
pls2p2 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt2")
pls2p3 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt3")
pls2p4 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt4")
pls2p5 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt5")
pls3p1 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt1")
pls3p2 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt2")
pls3p3 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt3")
pls3p4 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt4")
pls4p1 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p1")
pls4p2 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p2")
pls4p3 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p3")
pls4p4 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p4")
pls4p5 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p5")
pls4p6 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p6")
pls4p7 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p7")

# Planet Usable Data Masks
pls1p1_udm = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt1/swath1_pt1_udm")
pls2p1_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt1_udm")
pls2p2_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt2_udm")
pls2p3_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt3_udm")
pls2p4_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt4_udm")
pls2p5_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt2/swath2_pt5_udm")
pls3p1_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt1_udm")
pls3p2_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt2_udm")
pls3p3_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt3_udm")
pls3p4_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt3/swath3_pt4_udm")
pls4p1_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p1_udm")
pls4p2_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p2_udm")
pls4p3_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p3_udm")
pls4p4_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p4_udm")
pls4p5_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p5_udm")
pls4p6_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p6_udm")
pls4p7_udm  = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/pl/pt4/s4p7_udm")

# Reclassified VIIRS flood map we exported at the end of module 6
vfm = ee.Image(parent_directory + "vfm/vfm_mosaic")

# Reclassified GFM Flood Map we exported at the end of module 6
gfm = ee.Image(parent_directory + "gfm/gfm_mosaic")

# HYDRAFloods Map that we exported at the end of module X
hf = ee.Image(parent_directory + "hydrafloods/hf_mosaic")

# HYDROSAR Map that we exported at the end of Module 6
hs = ee.Image(parent_directory + "hydrosar/hydrosar_mosaic")

# Reclassified MCDWD Flood Map we exported at the end of module 6
mcdwd = ee.Image(parent_directory + "mcdwd/mcdwd_mosaic")

# Reclassified DSWx Flood Map we exported at the end of module 6
dswx = ee.Image(parent_directory + "dswx/dswx_mosaic")

# Step 3: Find the Area of Analysis

Before we proceed to the sampling design, we will find the area within the area of our interest where **no** optical sensor (e.g. Planet, MODIS, and HLS) has an unobstructed view of the ground.

## Step 3 Part 1: Planet

In [ ]:
# Planet UDM Mosaic
plum = ee.ImageCollection([pls1p1_udm, pls2p1_udm, pls2p2_udm, pls2p3_udm, pls2p4_udm, pls2p5_udm,
                           pls3p1_udm, pls3p2_udm, pls3p3_udm, pls3p4_udm, pls4p1_udm, pls4p2_udm,
                           pls4p3_udm, pls4p4_udm, pls4p5_udm, pls4p6_udm, pls4p7_udm]).max().clip(aoi)

# Planet Mosaic
plm = ee.ImageCollection([pls1p1, pls2p1, pls2p2, pls2p3, pls2p4, pls2p5, pls3p1, pls3p2, pls3p3,
                          pls3p4, pls4p1, pls4p2, pls4p3, pls4p4, pls4p5, pls4p6, pls4p7]).mosaic().clip(aoi)

# Planet Usable Data Mask Visualization Parameters
pluvp = {
    'bands': ['b1'],
    'min': 0,
    'max': 1,
    'palette': ['000000', 'FFFFFF']
}

# Planet Vizualization Parameters
plv = {
    'bands': ['b3', 'b2', 'b1'],
    'min': 0,
    'max': 2400,
    'gamma': 0.75
}

Visualize the Planet Imagery and Data Mask. The Data Mask will be visualized as white where we have usable data, and will appear black where we do not have usable data.

In [ ]:
aoi = ee.FeatureCollection(ee.String(parent_directory + 'aoi'))

# Get the coordinates of the center of the AOI for mapping purposes
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(aoi, {}, 'Area of interest')
Map.addLayer(plum, pluvp, 'Planet Usable Data Mask')
Map.addLayer(plm, plv, 'Planet Imagery')
Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

Planet Band Information: https://developers.planet.com/docs/data/udm-2/

In [ ]:
'''
clearfeats = clearobs.reduceToVectors(
    geometry=aoi.geometry(),
    geometryType='polygon',
    eightConnected=False,
    reducer=ee.Reducer.countEvery(),
    scale=3,
    maxPixels=1e14
)
'''

"\nclearfeats = clearobs.reduceToVectors(\n    geometry=aoi.geometry(),\n    geometryType='polygon',\n    eightConnected=False,\n    reducer=ee.Reducer.countEvery(),\n    scale=3,\n    maxPixels=1e14\n)\n"

In [ ]:
'''
listeder = clearfeats.toList(16000)
lengthen = listeder.length()
one = ee.Number(1)

ranger = lengthen.subtract(one)
ranger
'''

'\nlisteder = clearfeats.toList(16000)\nlengthen = listeder.length()\none = ee.Number(1)\n\nranger = lengthen.subtract(one)\nranger\n'

In [ ]:
'''
testlist_v3 = []
#mycount = 100

tolist = clearfeats.toList(count=lengthen)


for j in range(15919):
  thefeet = tolist.get(j)
  thegeom = thefeet.getInfo()['geometry']
  geomified = ee.Geometry(thegeom)
  #print(type(geomified))

  testlist_v3.append(geomified)

  if j % 100 == 0:
    print(j/15919 * 100, '% complete')

#feat = clearfeats.toList(count=20).get(12)
#print(feat.getInfo())
#testlist
#geomjson = feat.getInfo()['geometry']
#featgeom = feat.geometry()
'''

"\ntestlist_v3 = []\n#mycount = 100\n\ntolist = clearfeats.toList(count=lengthen)\n\n\nfor j in range(15919):\n  thefeet = tolist.get(j)\n  thegeom = thefeet.getInfo()['geometry']\n  geomified = ee.Geometry(thegeom)\n  #print(type(geomified))\n\n  testlist_v3.append(geomified)\n\n  if j % 100 == 0:\n    print(j/15919 * 100, '% complete')\n\n#feat = clearfeats.toList(count=20).get(12)\n#print(feat.getInfo())\n#testlist\n#geomjson = feat.getInfo()['geometry']\n#featgeom = feat.geometry()\n"

In [ ]:
#mpv14 = ee.Geometry.MultiPolygon(testlist_v3)

In [ ]:
#mpvfc = ee.FeatureCollection(mpv14)

In [ ]:
#geemap.ee_to_geojson(mpvfc, filename='multipolygon.geojson')

In [ ]:
'''
geemap.ee_export_vector_to_asset(
    ee.FeatureCollection(mpv14),
    description='Exporting_multipoly_V5',
    assetId='users/mickymags/flood_intercomparison/multipolygonv67_FINAL')
    '''

"\ngeemap.ee_export_vector_to_asset(\n    ee.FeatureCollection(mpv14),\n    description='Exporting_multipoly_V5',\n    assetId='users/mickymags/flood_intercomparison/multipolygonv67_FINAL')\n    "

In [ ]:
'''
testcoords = [
    [
        [
            [102.0, 2.0],
            [103.0, 2.0],
            [103.0, 3.0],
            [102.0, 3.0],
            [102.0, 2.0]
          ]
    ],
    [
        [
            [100.0, 0.0],
            [101.0, 0.0],
            [101.0, 1.0],
            [100.0, 0.0]
        ]
    ]
]
'''

'\ntestcoords = [\n    [\n        [\n            [102.0, 2.0],\n            [103.0, 2.0],\n            [103.0, 3.0],\n            [102.0, 3.0],\n            [102.0, 2.0]\n          ]\n    ],\n    [\n        [\n            [100.0, 0.0],\n            [101.0, 0.0],\n            [101.0, 1.0],\n            [100.0, 0.0]\n        ]\n    ]\n]\n'

In [ ]:
'''
geemap.ee_export_vector_to_asset(
    ee.FeatureCollection(mymp),
    description='Exporting Europe',
    assetId='users/mickymags/flood_intercomparison/multipolygon')
'''

"\ngeemap.ee_export_vector_to_asset(\n    ee.FeatureCollection(mymp),\n    description='Exporting Europe',\n    assetId='users/mickymags/flood_intercomparison/multipolygon')\n"

In [ ]:
'''
 {
   "type": "MultiPolygon",
   "coordinates": [
     [
       [
         [102.0, 2.0],
         [103.0, 2.0],
         [103.0, 3.0],
         [102.0, 3.0],
         [102.0, 2.0]
       ]
     ],
     [
       [
         [100.0, 0.0],
         [101.0, 0.0],
         [101.0, 1.0],
         [100.0, 1.0],
         [100.0, 0.0]
       ],
       [
         [100.2, 0.2],
         [100.2, 0.8],
         [100.8, 0.8],
         [100.8, 0.2],
         [100.2, 0.2]
       ]
     ]
   ]
 }
 '''

'\n {\n   "type": "MultiPolygon",\n   "coordinates": [\n     [\n       [\n         [102.0, 2.0],\n         [103.0, 2.0],\n         [103.0, 3.0],\n         [102.0, 3.0],\n         [102.0, 2.0]\n       ]\n     ],\n     [\n       [\n         [100.0, 0.0],\n         [101.0, 0.0],\n         [101.0, 1.0],\n         [100.0, 1.0],\n         [100.0, 0.0]\n       ],\n       [\n         [100.2, 0.2],\n         [100.2, 0.8],\n         [100.8, 0.8],\n         [100.8, 0.2],\n         [100.2, 0.2]\n       ]\n     ]\n   ]\n }\n '

In [ ]:
'''
feel = clearfeats.first()
first = feel.get(0)
geom = feel.geometry()
'''

'\nfeel = clearfeats.first()\nfirst = feel.get(0)\ngeom = feel.geometry()\n'

In [ ]:
'''
def exfun(thefeature):
  mygeom = thefeature.geometry()
  return mygeom
'''

'\ndef exfun(thefeature):\n  mygeom = thefeature.geometry()\n  return mygeom\n'

In [ ]:
#exfun(feel)

In [ ]:
#funcmapped = clearfeats.map(exfun)

In [ ]:
#funcmapped.getInfo()

In [ ]:
 #type(clearfeats)

In [ ]:
#cflist = clearfeats.toList(count = 1e6)

In [ ]:
#cflist

In [ ]:
'''
listed = cflist.getInfo()
lstlen = len(listed)
print(lstlen)

geom = ee.Geometry(listed[0]['geometry'])
lstr = ee.List(geom)
mpolytest = ee.Geometry.MultiPolygon(lstr)
mpolytest.getInfo()
'''

"\nlisted = cflist.getInfo()\nlstlen = len(listed)\nprint(lstlen)\n\ngeom = ee.Geometry(listed[0]['geometry'])\nlstr = ee.List(geom)\nmpolytest = ee.Geometry.MultiPolygon(lstr)\nmpolytest.getInfo()\n"

In [ ]:
'''
geomlist = []

for j in range(lstlen):
  geom = ee.Geometry(listed[j]['geometry'])
  geomlist.append(geom)

mpoly = ee.Geometry.MultiPolygon(geomlist)
mpoly.getInfo()
'''

"\ngeomlist = []\n\nfor j in range(lstlen):\n  geom = ee.Geometry(listed[j]['geometry'])\n  geomlist.append(geom)\n\nmpoly = ee.Geometry.MultiPolygon(geomlist)\nmpoly.getInfo()\n"

In [ ]:
'''
geemap.ee_export_vector_to_asset(
    ee.FeatureCollection(mpolytest),
    description='Exporting_multiploygon',
    assetId='users/mickymags/flood_intercomparison/multipolygon'
)
'''

"\ngeemap.ee_export_vector_to_asset(\n    ee.FeatureCollection(mpolytest),\n    description='Exporting_multiploygon',\n    assetId='users/mickymags/flood_intercomparison/multipolygon'\n)\n"

In [ ]:
'''
firstpoly = cflist.get(0).getInfo()
#firstcoords = firstpoly.get('coordinates')
#firstcoords.getInfo()
#firstpoly['coordinates']
coords = firstpoly['geometry']
thisismygeometry = ee.Geometry(coords)
#thisismygeometry.getInfo()
firstpoly
'''

"\nfirstpoly = cflist.get(0).getInfo()\n#firstcoords = firstpoly.get('coordinates')\n#firstcoords.getInfo()\n#firstpoly['coordinates']\ncoords = firstpoly['geometry']\nthisismygeometry = ee.Geometry(coords)\n#thisismygeometry.getInfo()\nfirstpoly\n"

In [ ]:
'''
def extractorfunc(myfeature):
  info = myfeature.getInfo()
  mycords = info['geometry']
  mygeom = ee.Geometry(mycords)

  return mygeom
'''

"\ndef extractorfunc(myfeature):\n  info = myfeature.getInfo()\n  mycords = info['geometry']\n  mygeom = ee.Geometry(mycords)\n\n  return mygeom\n"

In [ ]:
'''
firstfeat = cflist.get(0)
functest = extractorfunc(firstfeat)


funcmapped = cflist.map(extractorfunc)
'''

'\nfirstfeat = cflist.get(0)\nfunctest = extractorfunc(firstfeat)\n\n\nfuncmapped = cflist.map(extractorfunc)\n'

In [ ]:
#mpoly = ee.Geometry.MultiPolygon(cflist)

In [ ]:
#mpoly.getInfo()

In [ ]:
#clearfeats.getInfo()

In [ ]:
#combined = clearfeats.geometry().union()

## Step 1 Part 2: Get Masks for Optical Flood Map Products

In [ ]:
# This image will have a value of 1 where there is not a clear observation and a value of 0 where there is a clear observation
unclear = plum.select(['b1']).eq(0)

# This image will have a value of 1 where there is a clear observations
#clearobs = plum.select(['b1']).eq(1)

In [ ]:
################## Get masked area for each optical flood map ##################

# VIIRS Flood Map
vfm_mask = vfm.eq(2)       # VFM is the vfm map we exported at the end of module 6.

# Copernicus Flood Map
gfm_mask = gfm.eq(2)       # GFM is the GFM map we exported at the end of module 6.

# MCDWD Flood Map
mcdwd_mask = mcdwd.eq(2)

# DSWx Flood Map
dswx_mask = dswx.eq(2)


#Get spatial union where any of the flood maps are masked
final_mask = unclear.eq(1).Or(vfm_mask.eq(1)).Or(gfm_mask.eq(1)).Or(mcdwd_mask.eq(1)).Or(dswx_mask.eq(1))

Now we need to get the region where final_mask has a value of 0. This is the region we want to limit our analysis to.

In [ ]:
mask_vp = {
    'min': 0,
    'max': 1,
    'palette': ['000000', 'FFFFFF']
}

In [ ]:
ndwi_vp = {
    'min': -1,
    'max': 1
}

In [ ]:
mask2 = final_mask.eq(0)
pl_masked = plm.updateMask(mask2)

#rtv = final_mask.reduceToVectors(scale = 10, maxPixels = 1e13, geometry = aoi)

Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(aoi, {}, 'Area of Interest')
Map.addLayer(unclear, mask_vp, 'Planet Mask')
Map.addLayer(vfm_mask, mask_vp, 'VFM Mask')
Map.addLayer(gfm_mask, mask_vp, 'GFM Mask')
Map.addLayer(mcdwd_mask, mask_vp, 'MCDWD Mask')
Map.addLayer(dswx_mask, mask_vp, 'DSWx Mask')
Map.addLayer(final_mask, mask_vp, 'Final Mask')
Map.addLayer(pl_masked, plv, 'Planet Masked')
#Map.addLayer(ndwi, ndwi_vp, 'NDWI')
Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

Now let's export the final mask to Google Earth Engine, which will be used in the reference data collection stage

In [ ]:
# Define a function that exports an Image to a Google Earth Engine Asset.
def exporter(img, asset_id, scale_):

  desc = 'Flood_Map_Export_'
  #asset_id = parent_directory + 'Final_Cloud_Mask'
  region_ = aoi.geometry()
  geemap.ee_export_image_to_asset(image = img,
                                  assetId = asset_id,
                                  description = desc,
                                  region = region_,
                                  crs = 'EPSG:4326',
                                  scale = scale_,
                                  maxPixels = 1e13)
  return 0

In [ ]:
exporter(final_mask, parent_directory + 'final_mask', 30)

0

In [ ]:
#mf2 = maskfeatures.filter(ee.Filter.neq('b1', 0))

In [ ]:
#print(mf2.first().getInfo())

In [ ]:
#mf2u = mf2.union()

In [ ]:
#combine = maskfeatures.combine().geometry

In [ ]:
#type(mf2u)

 Export the masked Planet image to a Google Earth Engine Asset so we can use it in Collect Earth Online.

In [ ]:
pl_asset_id = parent_directory + 'pl/' + 'PL_mosaic'
pl_asset_description = 'planet_trucolor_mosaic_pk_061623'
geemap.ee_export_image_to_asset(image = pl_masked,
                                  assetId = pl_asset_id,
                                  description = pl_asset_description,
                                  #region = aoi,
                                  crs = 'EPSG:4326',
                                  scale = 3.7,
                                  maxPixels = 1e13)

Now let's create an NDWI image

In [ ]:
ee.Image(pls1p1).bandNames()

In [ ]:
pl_masked.bandNames()

b2 is green and b4 is nir. (McFeeters et al 2013](https://www.mdpi.com/2072-4292/5/7/3544) has the following equation

$ \frac{Green - NIR}{Green + NIR} $

In our planet image, green light is stored in b2 and near infrared light is stored in b4.

In [ ]:
band2 = pl_masked.select('b2')
band4 = pl_masked.select('b4')
ndwi = band2.subtract(band4).divide(band4.add(band2)).rename('NDWI')

In [ ]:
ndwi_vp = {
    'min': -1,
    'max': 1
}

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(aoi, {}, 'Area of Interest')
Map.addLayer(pl_masked, plv, 'Planet Masked')
Map.addLayer(ndwi, ndwi_vp, 'NDWI')
Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

Now let's export the NDWI map to our Google Earth Engine.

In [ ]:
exporter(ndwi, parent_directory + 'pl_ndwi', 3.7)

0

In [ ]:
ndwi.bandNames()

## Step 2: Sampling Design

Now we want to develop a stratified sampling design.

## Step 2 Part 1: Land Cover

We will use Dynamic World to stratify by land cover.

We want to get the Dynamic World Map from before the flood event.

In [ ]:
# Date at which the flood event starts
doi = "2023-06-16"

# Get the date from a week prior
week = ee.Date(doi).advance(-1, 'week')

# Get the date from six months prior
month6 = ee.Date(doi).advance(-6, 'month')

szn = ee.Date(doi).advance(-3, 'month')

In [ ]:
# Get the Dynamic World Image Collection
dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")


'''
# Filter the Dynamic World Image Collection to the area of interest and to the time period from a week before the flood event starts
dw_filt = dw.filterBounds(aoi).filterDate(week, doi)

# Get the dynamic world collection for the first day of the image collection
dwf = dw_filt.first()
date = ee.Date(dwf.get('system:time_start'))
date2 = date.advance(1, 'week')
dwf2 = dw.filterDate(date, date2).filterBounds(aoi)
'''
dw_filt = dw.filterBounds(aoi).filterDate(week, doi)

# Filter the Dynamic World Image Collection to the area of interest and to the time period from 6 months before the flood event starts
dw_filt_6 = dw.filterBounds(aoi).filterDate(month6, doi)

# Filter the Dynamic World Image to the beginning of the prior season
dw_filt_seasonal = dw.filterBounds(aoi).filterDate(szn, doi).select(['label'])

# Apply a temporal median reducer at each pixel to dw_filt_6
dw_redux_6 = dw_filt_6.mode()


dw_redux_sznl = dw_filt_seasonal.mode()

# Mosaic the Dynamic World Map
#dw_mos = dwf2.median().clip(aoi)

# Print the date for which the Dynamic World Map we are using was created
#date

In [ ]:
#dw_discrete = dw_mos.select(['label'])
#dw_masked = dw_discrete.updateMask(mask2)

# Select the 'label band', which contains discrete land cover classifications
# and clip the land cover image to the area of interest.
#dw_discrete_6 = dw_redux_6.select(['label']).clip(aoi)
dw_discrete_sznl = dw_redux_sznl.clip(aoi)
#dw_masked = dw_discrete.updateMask(mask2)

In [ ]:
dw_discrete_sznl.bandNames()

In [ ]:
#dw_d2 = dwf.select(['label']).clip(aoi).updateMask(mask2)

In [ ]:
dwd3 = dw_filt.mosaic().select(['label']).clip(aoi)

Visualize the Dynamic World Product

In [ ]:
dwvp = {
    'min': 0,
    'max': 8,
    'palette': ['419bdf', '397d49', '88b053', '7a87c6', 'e49635',
                'dfc35a', 'c4281b', 'a59b8f', 'b39fe1']
}
Map = geemap.Map(center = (lat, lon), zoom = 7)
#Map.addLayer(dwd3, dwvp, 'Dynamic World')
#Map.addLayer(dw_discrete_6, dwvp, '6 months land cover')
Map.addLayer(dw_discrete_sznl, dwvp, 'Seasonal land cover')
Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

Now, we want to amalgamate rare classes that are not of interest to us into a background class.

 **Our classes of particular interest are flooded vegetation, built-up, and bare ground, so we will not exclude these from the sampling design even if they are rare**. The reason these classes are of interest to us is because SAR flood mapping can result in false positives for bare ground and built-up, and

So let's see what land cover classes are rare in our area of interest.

Credit to Biplov Bhandari for the grouped reducer portion of the code below, check out his blog post [here](https://spatialthoughts.com/2020/06/19/calculating-area-gee/)

In [ ]:
#print(dwtest.getInfo())

In [ ]:
areaimg7 = ee.Image.pixelArea().addBands(dwd3)

dwtester = areaimg7.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': aoi.geometry(),
    'scale': 10,
    'maxPixels': 1e10
})

pixecountdic7 = dwtester.getInfo()
pixecountdic7

{'groups': [{'class': 0, 'sum': 716735793.6341518},
  {'class': 1, 'sum': 2492509826.9534497},
  {'class': 2, 'sum': 118732838.57571007},
  {'class': 3, 'sum': 56517592.20303224},
  {'class': 4, 'sum': 8185966048.215703},
  {'class': 5, 'sum': 1130670925.3324738},
  {'class': 6, 'sum': 4235790746.458164},
  {'class': 7, 'sum': 18643172808.399784},
  {'class': 8, 'sum': 10751205.10071061}]}

In [ ]:
# Use a grouped reducer to cound the number of pixels in each class
areaimg6 = ee.Image.pixelArea().addBands(dw_discrete_sznl)

dwgr = areaimg6.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': aoi.geometry(),
    'scale': 10,
    'maxPixels': 1e10
})

# Calculate the Total Number of pixels in the land cover map
pixcountdic = dwgr.getInfo()

groups = pixcountdic['groups']

mysum = 0

for j in range(len(groups)):
  mysum += groups[j]['sum']



# Calculate the Percentage of the area of interest occupied by each class
water_percent = groups[0]['sum'] / mysum * 100
trees_percent = groups[1]['sum'] / mysum * 100
grass_percent = groups[2]['sum'] / mysum * 100
flooded_veg_percent = groups[3]['sum'] / mysum * 100
crops_percent = groups[4]['sum'] / mysum * 100
shrub_percent = groups[5]['sum'] / mysum * 100
built_percent = groups[6]['sum'] / mysum * 100
bare_percent = groups[7]['sum'] / mysum * 100
snow_and_ice_percent = groups[8]['sum'] / mysum * 100

print('Water:',water_percent)
print('Trees:', trees_percent)
print('Grass: ',grass_percent)
print('Flooded Vegetation Percent:', flooded_veg_percent)
print('Cropland Percent:', crops_percent)
print('Shrub Percent: ', shrub_percent)
print('Built-up Percent: ', built_percent)
print('Bare Ground Percent: ', bare_percent)
print('Snow and Ice Percent: ', snow_and_ice_percent)

Water: 1.0012807724590116
Trees: 5.479364162508171
Grass:  0.16870041539594263
Flooded Vegetation Percent: 0.03122861651413496
Cropland Percent: 56.0441643640845
Shrub Percent:  3.8922719532281587
Built-up Percent:  9.89832664752535
Bare Ground Percent:  23.453045794110967
Snow and Ice Percent:  0.03161727417375972


Water, Grass, and Snow and Ice occupy less than 3% of our area of interest, and are not land cover classes of interest, so we will combine them into a background class. We will always include water as a member of the background class because we will have a representation of floodwater (in our next step).

In [ ]:
water = dw_discrete_sznl.eq(0)
trees = dw_discrete_sznl.eq(1)
grass = dw_discrete_sznl.eq(2)
flooded_vegetation = dw_discrete_sznl.eq(3)
crops = dw_discrete_sznl.eq(4)
shrub_and_scrub = dw_discrete_sznl.eq(5)
built = dw_discrete_sznl.eq(6)
bare = dw_discrete_sznl.eq(7)
snow_and_ice = dw_discrete_sznl.eq(8)

background_class = water.Or(grass).Or(flooded_vegetation).Or(snow_and_ice)

# Step 2 Part 2: Flood vs. Non-flood

In [ ]:
# Find where GFM is equal to 1
product_water = gfm.eq(1)

JRC dataset: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_YearlyHistory

for each year, has a 0 for no data, 1 for not water, 2 for seasonal water, and 3 for permanent water

In [ ]:
jrc = ee.ImageCollection("JRC/GSW1_4/YearlyHistory").filterDate("2016-01-01", "2022-01-01").limit(5, "system:time_start", False).map(
    lambda x: x.select("waterClass").eq(3).unmask(0).gt(0).clip(aoi)
)

In [ ]:
jrcv2 = ee.ImageCollection("JRC/GSW1_4/YearlyHistory").filterDate("1985-01-01", doi)

In [ ]:
jrc2021 = ee.Image(jrcv2.toList(50).get(-1))

In [ ]:
jrc2 = jrc2021.clip(aoi)

In [ ]:
wawavp = {'min': 0,
          'max': 1,
          'palette': ['000000', 'FFFFFF']
}

jrcvp = {'min': 0,                                                      # No data = 0 = gray
         'max': 3,                                                      # No water = 1 = red
         'palette': ['cccccc', 'ff0000', '99d9ea', '0000ff']}           # Seasonal water = 2 = light blue
                                                                        # Permanent water = 3 = dark blue


In [ ]:
# Unmask JRC
jrc_unmasked = jrc2.unmask() #

# Get the no data and no water values
jrc_nonwater = jrc_unmasked.lt(2)

# Get the values where GFM is 1 and jrc_nonwater is 0
testinggg = jrc_nonwater.eq(1)
flood = product_water.And(testinggg)

#jrc_msk = jrc2.mask().clip(aoi) # jrc_msk will have a value of 1 where jrc2 has data (i.e. a value of 0, 1, or 2), and will have a value of 0 where jrc2 is masked

#jrc_msked = jrc_msk.eq(1)

# Flood = pixels where GFM is 1 (GFM identified surface water) AND jrc_msk is 1 (jrc2 has data) and jrc2 has a value less than 2
#jrcnone = jrc2.eq(None)
#jrc_perm_water = jrc2.eq(3)
#jrc_seasonal_water = jrc2.eq(2)
#flood = gfm.eq(1).And(jrc_water).And(jrc_msked)

In [ ]:
# Visualize GFM Water versus JRC water

Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(product_water, wawavp, 'GFM Water')
#Map.addLayer(jrc2, jrcvp, 'JRC')
Map.addLayer(jrc_unmasked, jrcvp, 'JRC unmasked')
#Map.addLayer(jrc_perm_water, wawavp, 'JRC Permanent Water')
#Map.addLayer(jrc_seasonal_water, wawavp, 'JRC Seasonal Water')
Map.addLayer(jrc_nonwater, wawavp, 'JRC nonwater')
#Map.addLayer(jrc_msk, wawavp, 'JRC Mask')
Map.addLayer(flood, wawavp, 'Flood')
#Map.addLayer(flooded, wawavp, 'Flooded')
Map.addLayerControl()
Map


Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

# Conduct Stratified Sample

We want to do multistage stratified sampling, with two different layers of strata, and one cluster: Clouded areas. We will treat our cluster as a strata, resulting in three total stratas.

1. Flood vs. Non-flood
2. Land Cover
3. Cloud vs. Non-cloud

the flood raster has a value of 1 where it is flooded and a value of 0 where it is not flooded.

Land cover has a value

There are 6 land cover classes after the creation of a background class, 2 possible flood classes, and 2 possible cloud classes. This will result in a total of 24 strata, as 6 * 2 * 2 = 24. We will use the following schema for our



 Pixel Value| Land Cover Class |LC Value| Flood or Not | Flood Value | Cloud or Not| Cloud Value
---    | :---:                    | ---  |   --- | --- | --- | ---
0     |  Background    | 0 or 2 or 3 or 8 |Nonflooded | 0 | Not Cloud | 0
1     |  Background     | 0 or 2 or 3 or 8 |Flooded| 1 | Not Cloud | 0
2     |  Trees   |1 | Nonflooded |0| Not Cloud | 0
3     |  Trees  |1|  Flooded |1 | Not Cloud | 0
4     |  Crops  |4|  Nonflooded |0| Not Cloud | 0
5     |  Crops   |4|  Flooded | 1 | Not Cloud | 0
6     |  Shrub and Scrub |5| Nonflooded |0| Not Cloud| 0
7     |  Shrub and Scrub   |5|   Flooded |1| Not Cloud |0
8     |  Built  |6|   Nonflooded |0| Not Cloud |0
9     |  Built   |6|   Flooded |1| Not Cloud |0
10    |  Bare  |7| Nonflooded|0| Not Cloud |0
11    |  Bare  |7| Flooded|1| Not Cloud |0
12    |  Background  |0 or 2 or 3 or 8| Nonflooded|0| Cloud |1
13    |  Background   |0 or 2 or 3 or 8|Flooded|1| Cloud  |1
14    |  Trees   |1| Nonflooded|0| Cloud  |1
15    |  Trees   |1| Flooded |1| Cloud  |1
16    |  Crops  |4| Nonflooded |0|Cloud  |1
17    |  Crops |4| Flooded|1|Cloud |1
18    |  Shrub and Scrub   |5|Nonflooded |0|Cloud |1
19    |  Shrub and Scrub   |5|  Flooded |1| Cloud |1
20    |  Built    |6| Nonflooded |0|Cloud  |1
21    |  Built   |6|  Flooded |1| Cloud  |1
22    |  Bare   |7|   Nonflooded |0| Cloud  |1
23    |  Bare   |7|   Flooded|1| Cloud  | 1

In [ ]:
# Set up Logical Tests

flooded = flood.eq(1)
nonflooded = flood.eq(0)

cloudy = mask2.eq(1)
noncloudy = mask2.eq(0)

# Code the table above
s0 = background_class.And(nonflooded)#.And(noncloudy)
s1 = background_class.And(flooded)#.And(noncloudy)
s2 = trees.And(nonflooded)#.And(noncloudy)
s3 = trees.And(flooded)#.And(noncloudy)
s4 = crops.And(nonflooded)#.And(noncloudy)
s5 = crops.And(flooded)#.And(noncloudy)
s6 = shrub_and_scrub.And(nonflooded)#.And(noncloudy)
s7 = shrub_and_scrub.And(flooded)#.And(noncloudy)
s8 = built.And(nonflooded)#.And(noncloudy)
s9 = built.And(flooded)#.And(noncloudy)
s10 = bare.And(nonflooded)#.And(noncloudy)
s11 = bare.And(flooded)#.And(noncloudy)
#s12 = background_class.And(nonflooded).And(cloudy)
#s13 = background_class.And(flooded).And(cloudy)
#s14 = trees.And(nonflooded).And(cloudy)
#s15 = trees.And(flooded).And(cloudy)
#s16 = crops.And(nonflooded).And(cloudy)
#s17 = crops.And(flooded).And(cloudy)
#s18 = shrub_and_scrub.And(nonflooded).And(cloudy)
#s19 = shrub_and_scrub.And(flooded).And(cloudy)
#s20 = built.And(nonflooded).And(cloudy)
#s21 = built.And(flooded).And(cloudy)
#s22 = bare.And(nonflooded).And(cloudy)
#s23 = bare.And(flooded).And(cloudy)

In [ ]:
dwds_masked = dw_discrete_sznl.updateMask(mask2)
strata = dwds_masked.where(s0, ee.Image(0))
#strata = dw_discrete_sznl.where(s0, ee.Image(0))
#strata = ee.Image(0).clip(aoi)
strata = strata.where(s1, ee.Image(1))
strata = strata.where(s2, ee.Image(2))
strata = strata.where(s3, ee.Image(3))
strata = strata.where(s4, ee.Image(4))
strata = strata.where(s5, ee.Image(5))
strata = strata.where(s6, ee.Image(6))
strata = strata.where(s7, ee.Image(7))
strata = strata.where(s8, ee.Image(8))
strata = strata.where(s9, ee.Image(9))
strata = strata.where(s10, ee.Image(10))
strata = strata.where(s11, ee.Image(11))
#strata = strata.where(s12, ee.Image(12))
#strata = strata.where(s13, ee.Image(13))
#strata = strata.where(s14, ee.Image(14))
#strata = strata.where(s15, ee.Image(15))
#strata = strata.where(s16, ee.Image(16))
#strata = strata.where(s17, ee.Image(17))
#strata = strata.where(s18, ee.Image(18))
#strata = strata.where(s19, ee.Image(19))
#strata = strata.where(s20, ee.Image(20))
#strata = strata.where(s21, ee.Image(21))
#strata = strata.where(s22, ee.Image(22))
#strata = strata.where(s23, ee.Image(23))

# Delete the below cell after testing

In [ ]:
# Visualize GFM Water versus JRC water

Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(aoi, {}, 'Area of Interest')
Map.addLayer(flood, wawavp, 'Flood')
Map.addLayer(dwds_masked, dwvp, 'Seasonal land cover Masked')
Map.addLayer(final_mask, wawavp, 'final Mask')
Map.addLayer(s4, wawavp, 'Strata 4')
Map.addLayer(pl_masked, plv, 'Planet Masked')
Map.addLayer(nf_crops, {}, 'Class 4 Sample points')
#Map.addLayer(strahgeom, {}, 'Geometry')

Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
'''
areaimg3 = ee.Image.pixelArea().addBands(dwd3)

dwtest = areaimg3.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': aoi.geometry(),
    'scale': 10,
    'maxPixels': 1e10
})
'''

"\nareaimg3 = ee.Image.pixelArea().addBands(dwd3)\n\ndwtest = areaimg3.reduceRegion(**{\n    'reducer': ee.Reducer.sum().group(**{\n        'groupField': 1,\n        'groupName': 'class'\n    }),\n    'geometry': aoi.geometry(),\n    'scale': 10,\n    'maxPixels': 1e10\n})\n"

In [ ]:
'''
pixcountdic = dwtest.getInfo()
groups = pixcountdic['groups']

mysum = 0

for j in range(len(groups)):
  mysum += groups[j]['sum']
'''

"\npixcountdic = dwtest.getInfo()\ngroups = pixcountdic['groups']\n\nmysum = 0\n\nfor j in range(len(groups)):\n  mysum += groups[j]['sum']\n"

In [ ]:
'''
# Calculate the Percentage from each class

water_percent = groups[0]['sum'] / mysum * 100
trees_percent = groups[1]['sum'] / mysum * 100
grass_percent = groups[2]['sum'] / mysum * 100
flooded_veg_percent = groups[3]['sum'] / mysum * 100
crops_percent = groups[4]['sum'] / mysum * 100
shrub_percent = groups[5]['sum'] / mysum * 100
built_percent = groups[6]['sum'] / mysum * 100
bare_percent = groups[7]['sum'] / mysum * 100
snow_and_ice_percent = groups[8]['sum'] / mysum * 100

print('Water:',water_percent)
print('Trees:', trees_percent)
print('Grass: ',grass_percent)
print('Flooded Vegetation Percent:', flooded_veg_percent)
print('Cropland Percent:', crops_percent)
print('Shrub Percent: ', shrub_percent)
print('Built-up Percent: ', built_percent)
print('Bare Ground Percent: ', bare_percent)
print('Snow and Ice Percent: ', snow_and_ice_percent)
'''

"\n# Calculate the Percentage from each class\n\nwater_percent = groups[0]['sum'] / mysum * 100\ntrees_percent = groups[1]['sum'] / mysum * 100\ngrass_percent = groups[2]['sum'] / mysum * 100\nflooded_veg_percent = groups[3]['sum'] / mysum * 100\ncrops_percent = groups[4]['sum'] / mysum * 100\nshrub_percent = groups[5]['sum'] / mysum * 100\nbuilt_percent = groups[6]['sum'] / mysum * 100\nbare_percent = groups[7]['sum'] / mysum * 100\nsnow_and_ice_percent = groups[8]['sum'] / mysum * 100\n\nprint('Water:',water_percent)\nprint('Trees:', trees_percent)\nprint('Grass: ',grass_percent)\nprint('Flooded Vegetation Percent:', flooded_veg_percent)\nprint('Cropland Percent:', crops_percent)\nprint('Shrub Percent: ', shrub_percent)\nprint('Built-up Percent: ', built_percent)\nprint('Bare Ground Percent: ', bare_percent)\nprint('Snow and Ice Percent: ', snow_and_ice_percent)\n"

In [ ]:
'''
# DW = Dynamic World

s0 = background_class.And(nonflooded)                # DW Trees, Not Flood. These pixels will be assigned a value of 0
s1 = background_class.And(flooded)                   # DW Trees and Flood. These pixels will be assigned a value of 1.
s2 = flooded_vegetation.And(nonflooded)
s3 = flooded_vegetation.And(flooded)
s4 = crops.And(nonflooded)
s5 = crops.And(flooded)
s6 = shrub_and_scrub.And(nonflooded)
s7 = shrub_and_scrub.And(flooded)
s8 = built.And(nonflooded)
s9 = built.And(flooded)
s10 = bare.And(nonflooded)
s11 = bare.And(flooded)
s12 = trees.And(nonflooded)
s13 = trees.And(flooded)
'''

'\n# DW = Dynamic World\n\ns0 = background_class.And(nonflooded)                # DW Trees, Not Flood. These pixels will be assigned a value of 0\ns1 = background_class.And(flooded)                   # DW Trees and Flood. These pixels will be assigned a value of 1.\ns2 = flooded_vegetation.And(nonflooded)\ns3 = flooded_vegetation.And(flooded)\ns4 = crops.And(nonflooded)\ns5 = crops.And(flooded)\ns6 = shrub_and_scrub.And(nonflooded)\ns7 = shrub_and_scrub.And(flooded)\ns8 = built.And(nonflooded)\ns9 = built.And(flooded)\ns10 = bare.And(nonflooded)\ns11 = bare.And(flooded)\ns12 = trees.And(nonflooded)\ns13 = trees.And(flooded)\n'

In [ ]:
'''
# DW = Dynamic World

s0 = trees.And(nonflooded)                # DW Trees, Not Flood. These pixels will be assigned a value of 0
s1 = trees.And(flooded)                   # DW Trees and Flood. These pixels will be assigned a value of 1.
s2 = flooded_vegetation.And(nonflooded)
s3 = flooded_vegetation.And(flooded)
s4 = crops.And(nonflooded)
s5 = crops.And(flooded)
s6 = shrub_and_scrub.And(nonflooded)
s7 = shrub_and_scrub.And(flooded)
s8 = built.And(nonflooded)
s9 = built.And(flooded)
s10 = bare.And(nonflooded)
s11 = bare.And(flooded)
s12 = background_class.And(nonflooded)
s13 = background_class.And(flooded)
'''

'\n# DW = Dynamic World\n\ns0 = trees.And(nonflooded)                # DW Trees, Not Flood. These pixels will be assigned a value of 0\ns1 = trees.And(flooded)                   # DW Trees and Flood. These pixels will be assigned a value of 1.\ns2 = flooded_vegetation.And(nonflooded)\ns3 = flooded_vegetation.And(flooded)\ns4 = crops.And(nonflooded)\ns5 = crops.And(flooded)\ns6 = shrub_and_scrub.And(nonflooded)\ns7 = shrub_and_scrub.And(flooded)\ns8 = built.And(nonflooded)\ns9 = built.And(flooded)\ns10 = bare.And(nonflooded)\ns11 = bare.And(flooded)\ns12 = background_class.And(nonflooded)\ns13 = background_class.And(flooded)\n'

In [ ]:
strata_vp = {                         # Each land cover class will have two classes. The color will be light if it is not flooded on the date of interest,
    'min': 0,                         # and the color will be dark if it is flooded on the date of interest
    'max': 23,
    'palette': ['add8e6', '00008b',   # Background Class = Blue
                '90ee90', '06402b',   # Trees = Green
                'ffdbbb', 'ff8c00',   # Crops = Orange
                'ffffc5', '9b870c',   # Shrub and Scrub = Yellow
                'ffcccb', '8b0000',   # Built = Red
                'd3d3d3', '636363',   # Bare Ground = Gray
                'ffffff', 'ffffff', 'ffffff', 'ffffff','ffffff', 'ffffff',       # Cloudy Areas = white
                'ffffff', 'ffffff','ffffff', 'ffffff','ffffff', 'ffffff']
}

Let's visualize the old land cover map and the new Strata.

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(dw_discrete_sznl, dwvp, 'Dynamic World')
Map.addLayer(final_mask, wawavp, 'final Mask')
Map.addLayer(strata, strata_vp, 'Strata')
Map.addLayer(gfm, wawavp, 'GFM')
Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

Sanity Check

In [ ]:
areaimg4 = ee.Image.pixelArea().addBands(strata)

strata_test = areaimg4.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': aoi.geometry(),
    'scale': 10,
    'maxPixels': 1e10
})

In [ ]:
pcdic = strata_test.getInfo()
groups2 = pcdic['groups']

mysum2 = 0

for k in range(len(groups2)):
  mysum2 += groups2[k]['sum']

In [ ]:
pcdic

{'groups': [{'class': 0, 'sum': 313562984.70049286},
  {'class': 1, 'sum': 19444691.291397095},
  {'class': 2, 'sum': 156054407.06761932},
  {'class': 3, 'sum': 81422.80867004395},
  {'class': 4, 'sum': 13908508303.279015},
  {'class': 5, 'sum': 120009778.6180191},
  {'class': 6, 'sum': 136239069.78762817},
  {'class': 7, 'sum': 747124.5773620605},
  {'class': 8, 'sum': 2269993172.954399},
  {'class': 9, 'sum': 1042972.587097168},
  {'class': 10, 'sum': 967126044.9537888},
  {'class': 11, 'sum': 21304719.071113586}]}

{'groups': [{'class': 0, 'sum': 313562984.70049286},
  {'class': 1, 'sum': 19444691.291397095},
  {'class': 2, 'sum': 156054407.06761932},
  {'class': 3, 'sum': 81422.80867004395},
  {'class': 4, 'sum': 13908508303.279015},
  {'class': 5, 'sum': 120009778.6180191},
  {'class': 6, 'sum': 136239069.78762817},
  {'class': 7, 'sum': 747124.5773620605},
  {'class': 8, 'sum': 2269993172.954399},
  {'class': 9, 'sum': 1042972.587097168},
  {'class': 10, 'sum': 967126044.9537888},
  {'class': 11, 'sum': 21304719.071113586}]}

In [ ]:
w0 = pcdic['groups'][0]['sum'] / mysum2
w1 = pcdic['groups'][1]['sum'] / mysum2 # Strata 1 Area as a proportion to total area
w2 = pcdic['groups'][2]['sum'] / mysum2
w3 = pcdic['groups'][3]['sum'] / mysum2
w4 = pcdic['groups'][4]['sum'] / mysum2
w5 = pcdic['groups'][5]['sum'] / mysum2
w6 = pcdic['groups'][6]['sum'] / mysum2
w7 = pcdic['groups'][7]['sum'] / mysum2
w8 = pcdic['groups'][8]['sum'] / mysum2
w9 = pcdic['groups'][9]['sum'] / mysum2
w10 = pcdic['groups'][10]['sum'] / mysum2
w11 = pcdic['groups'][11]['sum'] / mysum2
#w12 = pcdic['groups'][12]['sum'] / mysum2
#w13 = pcdic['groups'][13]['sum'] / mysum2
#w14 = pcdic['groups'][14]['sum'] / mysum2
#w15 = pcdic['groups'][15]['sum'] / mysum2
#w16 = pcdic['groups'][16]['sum'] / mysum2
#w17 = pcdic['groups'][17]['sum'] / mysum2
#w18 = pcdic['groups'][18]['sum'] / mysum2
#w19 = pcdic['groups'][19]['sum'] / mysum2
#w20 = pcdic['groups'][20]['sum'] / mysum2
#w21 = pcdic['groups'][21]['sum'] / mysum2
#w22 = pcdic['groups'][22]['sum'] / mysum2
#w23 = pcdic['groups'][23]['sum'] / mysum2

In [ ]:
'''# s0 is the area that is unclassified by the Dynamic World Landcover Map

s0 = dwd3.eq(0).And(flood.eq(0))           # DW Water, Not Flood                           light blue
s1 = dwd3.eq(0).And(product_water.eq(1))   # DW Water, Prod                       dark blue
s2 = dwd3.eq(1).And(product_water.eq(0))   # DW Trees, Product Nonwater                    light green
s3 = dwd3.eq(1).And(product_water.eq(1))   # DW Trees, Product Water                       dark green
s4 = dwd3.eq(2).And(product_water.eq(0))   # DW Grass, Product Nonwater                    light gray
s5 = dwd3.eq(2).And(product_water.eq(1))   # DW Grass, Product Water                       dark gray
s6 = dwd3.eq(3).And(product_water.eq(0))   # DW Flooded Vegetation, Product Nonwater       light purple
s7 = dwd3.eq(3).And(product_water.eq(1))   # DW Flooded Vegetation, Product Water          dark purple
s8 = dwd3.eq(4).And(product_water.eq(0))   # DW Crops, Product Nonwater                    light orange
s9 = dwd3.eq(4).And(product_water.eq(1))  # DW Crops, Product Water                       dark orange
s10 = dwd3.eq(5).And(product_water.eq(0))  # DW Shrub and Scrub, Product Nonwater          light yellow
s11 = dwd3.eq(5).And(product_water.eq(1))  # DW Shrub and Scrub, Product Water             dark yellow
s12 = dwd3.eq(6).And(product_water.eq(0))  # DW Built, Product Nonwater                    light red
s13 = dwd3.eq(6).And(product_water.eq(1))  # DW Built, Product Water                       dark red
s14 = dwd3.eq(7).And(product_water.eq(0))  # DW Bare, Product Nonwater                     light brown
s15 = dwd3.eq(7).And(product_water.eq(1))  # DW Bare, Product Water                        dark brown
s16 = dwd3.eq(8).And(product_water.eq(0))  # DW Snow and Ice, Product Nonwater             light tan
s17 = dwd3.eq(8).And(product_water.eq(1))  # DW Snow and Ice, Product Water                dark tan
'''

'# s0 is the area that is unclassified by the Dynamic World Landcover Map\n\ns0 = dwd3.eq(0).And(flood.eq(0))           # DW Water, Not Flood                           light blue\ns1 = dwd3.eq(0).And(product_water.eq(1))   # DW Water, Prod                       dark blue\ns2 = dwd3.eq(1).And(product_water.eq(0))   # DW Trees, Product Nonwater                    light green\ns3 = dwd3.eq(1).And(product_water.eq(1))   # DW Trees, Product Water                       dark green\ns4 = dwd3.eq(2).And(product_water.eq(0))   # DW Grass, Product Nonwater                    light gray\ns5 = dwd3.eq(2).And(product_water.eq(1))   # DW Grass, Product Water                       dark gray\ns6 = dwd3.eq(3).And(product_water.eq(0))   # DW Flooded Vegetation, Product Nonwater       light purple\ns7 = dwd3.eq(3).And(product_water.eq(1))   # DW Flooded Vegetation, Product Water          dark purple\ns8 = dwd3.eq(4).And(product_water.eq(0))   # DW Crops, Product Nonwater                    light orange\ns

'# s0 is the area that is unclassified by the Dynamic World Landcover Map\n\ns0 = dwd3.eq(0).And(flood.eq(0))           # DW Water, Not Flood                           light blue\ns1 = dwd3.eq(0).And(product_water.eq(1))   # DW Water, Prod                       dark blue\ns2 = dwd3.eq(1).And(product_water.eq(0))   # DW Trees, Product Nonwater                    light green\ns3 = dwd3.eq(1).And(product_water.eq(1))   # DW Trees, Product Water                       dark green\ns4 = dwd3.eq(2).And(product_water.eq(0))   # DW Grass, Product Nonwater                    light gray\ns5 = dwd3.eq(2).And(product_water.eq(1))   # DW Grass, Product Water                       dark gray\ns6 = dwd3.eq(3).And(product_water.eq(0))   # DW Flooded Vegetation, Product Nonwater       light purple\ns7 = dwd3.eq(3).And(product_water.eq(1))   # DW Flooded Vegetation, Product Water          dark purple\ns8 = dwd3.eq(4).And(product_water.eq(0))   # DW Crops, Product Nonwater                    light orange\ns

In [ ]:
'''
new_classes = ee.Image(0).clip(aoi)

new_classes2 = new_classes.where(s1, ee.Image(1))
new_classes3 = new_classes2.where(s2, ee.Image(2))
new_classes4 = new_classes3.where(s3, ee.Image(3))
new_classes5 = new_classes4.where(s4, ee.Image(4))
new_classes6 = new_classes5.where(s5, ee.Image(5))
new_classes7 = new_classes6.where(s6, ee.Image(6))
new_classes8 = new_classes7.where(s7, ee.Image(7))
new_classes9 = new_classes8.where(s8, ee.Image(8))
new_classes10 = new_classes9.where(s9, ee.Image(9))
new_classes11 = new_classes10.where(s10, ee.Image(10))
new_classes12 = new_classes11.where(s11, ee.Image(11))
new_classes13 = new_classes12.where(s12, ee.Image(12))
new_classes14= new_classes13.where(s13, ee.Image(13))
new_classes15 = new_classes14.where(s14, ee.Image(14))
new_classes16 = new_classes15.where(s15, ee.Image(15))
new_classes17 = new_classes16.where(s16, ee.Image(16))
new_classes18 = new_classes17.where(s17, ee.Image(17))
new_classes19 = new_classes18.where(s18, ee.Image(18))
new_classes20 = new_classes19.where(s19, ee.Image(19))
new_classes21 = new_classes20.where(s20, ee.Image(20))
'''

'\nnew_classes = ee.Image(0).clip(aoi)\n\nnew_classes2 = new_classes.where(s1, ee.Image(1))\nnew_classes3 = new_classes2.where(s2, ee.Image(2))\nnew_classes4 = new_classes3.where(s3, ee.Image(3))\nnew_classes5 = new_classes4.where(s4, ee.Image(4))\nnew_classes6 = new_classes5.where(s5, ee.Image(5))\nnew_classes7 = new_classes6.where(s6, ee.Image(6))\nnew_classes8 = new_classes7.where(s7, ee.Image(7))\nnew_classes9 = new_classes8.where(s8, ee.Image(8))\nnew_classes10 = new_classes9.where(s9, ee.Image(9))\nnew_classes11 = new_classes10.where(s10, ee.Image(10))\nnew_classes12 = new_classes11.where(s11, ee.Image(11))\nnew_classes13 = new_classes12.where(s12, ee.Image(12))\nnew_classes14= new_classes13.where(s13, ee.Image(13))\nnew_classes15 = new_classes14.where(s14, ee.Image(14))\nnew_classes16 = new_classes15.where(s15, ee.Image(15))\nnew_classes17 = new_classes16.where(s16, ee.Image(16))\nnew_classes18 = new_classes17.where(s17, ee.Image(17))\nnew_classes19 = new_classes18.where(s18, ee

'\nnew_classes = ee.Image(0).clip(aoi)\n\nnew_classes2 = new_classes.where(s1, ee.Image(1))\nnew_classes3 = new_classes2.where(s2, ee.Image(2))\nnew_classes4 = new_classes3.where(s3, ee.Image(3))\nnew_classes5 = new_classes4.where(s4, ee.Image(4))\nnew_classes6 = new_classes5.where(s5, ee.Image(5))\nnew_classes7 = new_classes6.where(s6, ee.Image(6))\nnew_classes8 = new_classes7.where(s7, ee.Image(7))\nnew_classes9 = new_classes8.where(s8, ee.Image(8))\nnew_classes10 = new_classes9.where(s9, ee.Image(9))\nnew_classes11 = new_classes10.where(s10, ee.Image(10))\nnew_classes12 = new_classes11.where(s11, ee.Image(11))\nnew_classes13 = new_classes12.where(s12, ee.Image(12))\nnew_classes14= new_classes13.where(s13, ee.Image(13))\nnew_classes15 = new_classes14.where(s14, ee.Image(14))\nnew_classes16 = new_classes15.where(s15, ee.Image(15))\nnew_classes17 = new_classes16.where(s16, ee.Image(16))\nnew_classes18 = new_classes17.where(s17, ee.Image(17))\nnew_classes19 = new_classes18.where(s18, ee

In [ ]:
#final_strata = new_classes21.updateMask(mask2)

In [ ]:
'''
Map = geemap.Map(center = (lat, lon), zoom = 7)
Map.addLayer(new_classes21, newvp, 'New Land Cover')
Map.addLayer(final_strata, newvp, 'Final strata ? ')
Map.addLayerControl()
Map
'''

"\nMap = geemap.Map(center = (lat, lon), zoom = 7)\nMap.addLayer(new_classes21, newvp, 'New Land Cover')\nMap.addLayer(final_strata, newvp, 'Final strata ? ')\nMap.addLayerControl()\nMap\n"

"\nMap = geemap.Map(center = (lat, lon), zoom = 7)\nMap.addLayer(new_classes21, newvp, 'New Land Cover')\nMap.addLayer(final_strata, newvp, 'Final strata ? ')\nMap.addLayerControl()\nMap\n"

In [ ]:
'''
areaimg = ee.Image.pixelArea().addBands(new_classes21)

areas = areaimg.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': aoi.geometry(),
    'scale': 10,
    'maxPixels': 1e10
})
'''

"\nareaimg = ee.Image.pixelArea().addBands(new_classes21)\n\nareas = areaimg.reduceRegion(**{\n    'reducer': ee.Reducer.sum().group(**{\n        'groupField': 1,\n        'groupName': 'class'\n    }),\n    'geometry': aoi.geometry(),\n    'scale': 10,\n    'maxPixels': 1e10\n})\n"

"\nareaimg = ee.Image.pixelArea().addBands(new_classes21)\n\nareas = areaimg.reduceRegion(**{\n    'reducer': ee.Reducer.sum().group(**{\n        'groupField': 1,\n        'groupName': 'class'\n    }),\n    'geometry': aoi.geometry(),\n    'scale': 10,\n    'maxPixels': 1e10\n})\n"

In [ ]:
#areas.getInfo()

In [ ]:
'''
class1area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(0)).get('sum')).divide(1e6)   # Area of Class 1 in Square Kilometers
class2area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(1)).get('sum')).divide(1e6)   # Area of Class 2 in Square Kilometers
class3area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(2)).get('sum')).divide(1e6)   # Area of Class 3 in Square Kilometers
class4area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(3)).get('sum')).divide(1e6)   # Area of Class 4 in Square Kilometers
class5area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(4)).get('sum')).divide(1e6)   # Area of Class 5 in Square Kilometers
class6area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(5)).get('sum')).divide(1e6)   # Area of Class 6 in Square Kilometers
class7area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(6)).get('sum')).divide(1e6)   # Area of Class 7 in Square Kilometers
class8area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(7)).get('sum')).divide(1e6)   # Area of Class 8 in Square Kilometers
class9area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(8)).get('sum')).divide(1e6)   # Area of Class 9 in Square Kilometers
class10area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(9)).get('sum')).divide(1e6)   # Area of Class 10 in Square Kilometers
class11area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(10)).get('sum')).divide(1e6)   # Area of Class 11 in Square Kilometers
class12area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(11)).get('sum')).divide(1e6)   # Area of Class 12 in Square Kilometers
class13area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(12)).get('sum')).divide(1e6)   # Area of Class 13 in Square Kilometers
class14area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(13)).get('sum')).divide(1e6)   # Area of Class 14 in Square Kilometers
class15area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(14)).get('sum')).divide(1e6)   # Area of Class 15 in Square Kilometers
class16area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(15)).get('sum')).divide(1e6)   # Area of Class 16 in Square Kilometers
class17area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(16)).get('sum')).divide(1e6)   # Area of Class 17 in Square Kilometers
class18area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(17)).get('sum')).divide(1e6)   # Area of Class 18 in Square Kilometers
class19area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(18)).get('sum')).divide(1e6)   # Area of Class 19 in Square Kilometers
class20area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(19)).get('sum')).divide(1e6)   # Area of Class 20 in Square Kilometers

sumclass = class1area.add(class2area).add(class3area).add(class4area)\
           .add(class5area).add(class6area).add(class7area).add(class8area)\
           .add(class9area).add(class10area).add(class11area).add(class12area)\
           .add(class13area).add(class14area).add(class15area).add(class16area)\
           .add(class17area).add(class18area).add(class19area).add(class20area)
'''


"\nclass1area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(0)).get('sum')).divide(1e6)   # Area of Class 1 in Square Kilometers\nclass2area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(1)).get('sum')).divide(1e6)   # Area of Class 2 in Square Kilometers\nclass3area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(2)).get('sum')).divide(1e6)   # Area of Class 3 in Square Kilometers\nclass4area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(3)).get('sum')).divide(1e6)   # Area of Class 4 in Square Kilometers\nclass5area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(4)).get('sum')).divide(1e6)   # Area of Class 5 in Square Kilometers\nclass6area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(5)).get('sum')).divide(1e6)   # Area of Class 6 in Square Kilometers\nclass7area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(6)).get('sum')).divide(1e6)   # Area of Class 7 in Square Kilometers\nclass8area = ee.N

"\nclass1area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(0)).get('sum')).divide(1e6)   # Area of Class 1 in Square Kilometers\nclass2area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(1)).get('sum')).divide(1e6)   # Area of Class 2 in Square Kilometers\nclass3area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(2)).get('sum')).divide(1e6)   # Area of Class 3 in Square Kilometers\nclass4area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(3)).get('sum')).divide(1e6)   # Area of Class 4 in Square Kilometers\nclass5area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(4)).get('sum')).divide(1e6)   # Area of Class 5 in Square Kilometers\nclass6area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(5)).get('sum')).divide(1e6)   # Area of Class 6 in Square Kilometers\nclass7area = ee.Number(ee.Dictionary(ee.List(areas.get('groups')).get(6)).get('sum')).divide(1e6)   # Area of Class 7 in Square Kilometers\nclass8area = ee.N

In [ ]:
'''
# Let w0 be the proportion of the area occupied by class0
w1 = class1area.divide(sumclass).getInfo()
w2 = class2area.divide(sumclass).getInfo()
w3 = class3area.divide(sumclass).getInfo()
w4 = class4area.divide(sumclass).getInfo()
w5 = class5area.divide(sumclass).getInfo()
w6 = class6area.divide(sumclass).getInfo()
w7 = class7area.divide(sumclass).getInfo()
w8 = class8area.divide(sumclass).getInfo()
w9 = class9area.divide(sumclass).getInfo()
w10 = class10area.divide(sumclass).getInfo()
w11 = class11area.divide(sumclass).getInfo()
w12 = class12area.divide(sumclass).getInfo()
w13 = class13area.divide(sumclass).getInfo()
w14 = class14area.divide(sumclass).getInfo()
w15 = class15area.divide(sumclass).getInfo()
w16 = class16area.divide(sumclass).getInfo()
w17 = class17area.divide(sumclass).getInfo()
w18 = class18area.divide(sumclass).getInfo()
w19 = class19area.divide(sumclass).getInfo()
w20 = class20area.divide(sumclass).getInfo()



print('Class 1 Area: ', class1area.getInfo(), 'Class 1 proportion:', w1)
print('Class 2 Area: ', class2area.getInfo(), 'Class 2 proportion:', w2)
print('Class 3 Area: ', class3area.getInfo(), 'Class 3 proportion:', w3)
print('Class 4 Area: ', class4area.getInfo(), 'Class 4 proportion:', w4)
print('Class 5 Area: ', class5area.getInfo(), 'Class 5 proportion:', w5)
print('Class 6 Area: ', class6area.getInfo(), 'Class 6 proportion:', w6)
print('Class 7 Area: ', class7area.getInfo(), 'Class 7 proportion:', w7)
print('Class 8 Area: ', class8area.getInfo(), 'Class 8 proportion:', w8)
print('Class 9 Area: ', class9area.getInfo(), 'Class 9 proportion:', w9)
print('Class 10 Area: ', class10area.getInfo(), 'Class 10 proportion:', w10)
print('Class 11 Area: ', class11area.getInfo(), 'Class 11 proportion:', w11)
print('Class 12 Area: ', class12area.getInfo(), 'Class 12 proportion:', w12)
print('Class 13 Area: ', class13area.getInfo(), 'Class 13 proportion:', w13)
print('Class 14 Area: ', class14area.getInfo(), 'Class 14 proportion:', w14)
print('Class 15 Area: ', class15area.getInfo(), 'Class 15 proportion:', w15)
print('Class 16 Area: ', class16area.getInfo(), 'Class 16 proportion:', w16)
print('Class 17 Area: ', class17area.getInfo(), 'Class 17 proportion:', w17)
print('Class 18 Area: ', class18area.getInfo(), 'Class 18 proportion:', w18)
print('Class 19 Area: ', class19area.getInfo(), 'Class 19 proportion:', w19)
print('Class 20 Area: ', class20area.getInfo(), 'Class 20 proportion:', w20)
'''

"\n# Let w0 be the proportion of the area occupied by class0\nw1 = class1area.divide(sumclass).getInfo()\nw2 = class2area.divide(sumclass).getInfo()\nw3 = class3area.divide(sumclass).getInfo()\nw4 = class4area.divide(sumclass).getInfo()\nw5 = class5area.divide(sumclass).getInfo()\nw6 = class6area.divide(sumclass).getInfo()\nw7 = class7area.divide(sumclass).getInfo()\nw8 = class8area.divide(sumclass).getInfo()\nw9 = class9area.divide(sumclass).getInfo()\nw10 = class10area.divide(sumclass).getInfo()\nw11 = class11area.divide(sumclass).getInfo()\nw12 = class12area.divide(sumclass).getInfo()\nw13 = class13area.divide(sumclass).getInfo()\nw14 = class14area.divide(sumclass).getInfo()\nw15 = class15area.divide(sumclass).getInfo()\nw16 = class16area.divide(sumclass).getInfo()\nw17 = class17area.divide(sumclass).getInfo()\nw18 = class18area.divide(sumclass).getInfo()\nw19 = class19area.divide(sumclass).getInfo()\nw20 = class20area.divide(sumclass).getInfo()\n\n\n\nprint('Class 1 Area: ', class1

"\n# Let w0 be the proportion of the area occupied by class0\nw1 = class1area.divide(sumclass).getInfo()\nw2 = class2area.divide(sumclass).getInfo()\nw3 = class3area.divide(sumclass).getInfo()\nw4 = class4area.divide(sumclass).getInfo()\nw5 = class5area.divide(sumclass).getInfo()\nw6 = class6area.divide(sumclass).getInfo()\nw7 = class7area.divide(sumclass).getInfo()\nw8 = class8area.divide(sumclass).getInfo()\nw9 = class9area.divide(sumclass).getInfo()\nw10 = class10area.divide(sumclass).getInfo()\nw11 = class11area.divide(sumclass).getInfo()\nw12 = class12area.divide(sumclass).getInfo()\nw13 = class13area.divide(sumclass).getInfo()\nw14 = class14area.divide(sumclass).getInfo()\nw15 = class15area.divide(sumclass).getInfo()\nw16 = class16area.divide(sumclass).getInfo()\nw17 = class17area.divide(sumclass).getInfo()\nw18 = class18area.divide(sumclass).getInfo()\nw19 = class19area.divide(sumclass).getInfo()\nw20 = class20area.divide(sumclass).getInfo()\n\n\n\nprint('Class 1 Area: ', class1

Define the estimated user's accuracy for each class. The user's accuracy is estimated to be the different for product water and nonwater, but the same across different land cover types. The estimates for user's accuracy for the water and nonwater classes were obtained based on reported statistics for each flood product. More details in this document: https://docs.google.com/document/d/1TBuHX-W2an7E4tvUVOC6RoRBInOPxnoVDRtqYHHn8y8/edit?usp=sharing.

Let $U_i$ be the user's accuracy for class i.

Let $U_{water}$ be the estimated user's accuracy for the classes that have product water (even-numbered classes)


$ U_{nonwater} $ be the estimated user's accuracy for the classes that have product nonwater

$U_{water} $ = 0.89448485408 \
$U_{nonwater}$ = 0.98517


---


UA nonwater, estimate = 0.98517


In [ ]:
u_water = 0.89448485408
u_nonwater = 0.98517

In [ ]:
u0 = u_nonwater  # User's Accuracy for Strata 0
u1 = u_water     # User's accuracy for Strata 1
u2 = u_nonwater     # User's Accuracy for Strata 2
u3 = u_water  # User's Accuracy for Strata 3
u4 = u_nonwater     # User's Accuracy for Strata 4
u5 = u_water  # User's Accuracy for Strata 5
u6 = u_nonwater     # User's Accuracy for Strata 6
u7 = u_water  # User's Accuracy for Strata 7
u8 = u_nonwater     # User's Accuracy for Strata 8
u9 = u_water  # User's Accuracy for Strata 9
u10 = u_nonwater    # User's Accuracy for Strata 10
u11 = u_water # User's Accuracy for Strata 11
#u12 = u_water    # User's Accuracy for Strata 12
#u13 = u_nonwater # User's Accuracy for Strata 13
#u14 = u_water    # User's Accuracy for Strata 14
#u15 = u_nonwater # User's Accuracy for Strata 15
#u16 = u_water    # User's Accuracy for Strata 16
#u17 = u_nonwater # User's Accuracy for Strata 17
#u18 = u_water    # User's Accuracy for Strata 18
#u19 = u_nonwater # User's Accuracy for strata 19
#u20 = u_water    # User's Accuracy for Strata 20

According to Olofsson, the standard deviation for class i $(S_i)$ can be calulated by

$$
S_i = \sqrt{U_i (1 - U_i)}
$$

Define a function that calculates the standard deviation of each stratum.

In [ ]:
def stdev(ui):
  term2 = 1 - ui
  term = ui * term2
  si = np.sqrt(term)
  return si

In [ ]:
sd0 = stdev(u0)
sd1 = stdev(u1)
sd2 = stdev(u2)
sd3 = stdev(u3)
sd4 = stdev(u4)
sd5 = stdev(u5)
sd6 = stdev(u6)
sd7 = stdev(u7)
sd8 = stdev(u8)
sd9 = stdev(u9)
sd10 = stdev(u10)
sd11 = stdev(u11)
#sd12 = stdev(u12)
#sd13 = stdev(u13)
#sd14 = stdev(u14)
#sd15 = stdev(u15)
#sd16 = stdev(u16)
#sd17 = stdev(u17)
#sd18 = stdev(u18)
#sd19 = stdev(u19)
#sd20 = stdev(u20)

To estimate the sample size, we must calculate the product of $W_i$ and $S_i$. Let's call this term $P_i$

In [ ]:
p0 = w0 * sd0
p1 = w1 * sd1
p2 = w2 * sd2
p3 = w3 * sd3
p4 = w4 * sd4
p5 = w5 * sd5
p6 = w6 * sd6
p7 = w7 * sd7
p8 = w8 * sd8
p9 = w9 * sd9
p10 = w10 * sd10
p11 = w11 * sd11
#p12 = w12 * sd12
#p13 = w13 * sd13
#p14 = w14 * sd14
#p15 = w15 * sd15
#p16 = w16 * sd16
#p17 = w17 * sd18
#p18 = w18 * sd18
#p19 = w19 * sd19
#p20 = w20 * sd20

In [ ]:
psum = p0 + p1 + p2 + p3 + p4 + p5 + p6 + p8 + p9 + p10 + p11 #+ p12 + p13 + p14 + p15 + p16 + p17 + p18 + p19 + p20
psum

0.1225510107650266

0.1225510107650266

In [ ]:
error0 = 1e-3
error1 = 2.5e-3
error2 = 5e-3
error3 = 7e-3
error4 = 1e-2
error5 = 5e-2

In [ ]:
# Sample Size Estimation Function
def sse(error):
  return np.round((psum/error)**2)

In [ ]:
ss0 = sse(error0)
ss1 = sse(error1)
ss2 = sse(error2)
ss3 = sse(error3)
ss4 = sse(error4)
ss5 = sse(error5)

In [ ]:
print("Using a standard error of {}, our total sample size would be {}".format(error0, ss0))
print("Using a standard error of {}, our total sample size would be {}".format(error1, ss1))
print("Using a standard error of {}, our total sample size would be {}".format(error2, ss2))
print("Using a standard error of {}, our total sample size would be {}".format(error3, ss3))
print("Using a standard error of {}, our total sample size would be {}".format(error4, ss4))

Using a standard error of 0.001, our total sample size would be 15019.0
Using a standard error of 0.0025, our total sample size would be 2403.0
Using a standard error of 0.005, our total sample size would be 601.0
Using a standard error of 0.007, our total sample size would be 307.0
Using a standard error of 0.01, our total sample size would be 150.0


Using a standard error of 0.001, our total sample size would be 15019.0
Using a standard error of 0.0025, our total sample size would be 2403.0
Using a standard error of 0.005, our total sample size would be 601.0
Using a standard error of 0.007, our total sample size would be 307.0
Using a standard error of 0.01, our total sample size would be 150.0


Let's use the sample size of 695. This is the total number of points we will use in our study area. Now we have to decide how to distribute these points within our study area.

One option is to use equal allocation, where we assign an equal number of points across our strata.

Another option is to use proportional allocation, where we assign a number of points to each strata proportional to the area occupied by that strata

A third option is to assign a certain number of points to each strata, and then assign the remainder of points according to a proportional distribution.


### Equal Allocation

In [ ]:
equal = ss2 / 12

In [ ]:
equal

50.083333333333336

50.083333333333336

### Proportional Allocation

In [ ]:
n0 = np.round(w0 * ss1)
n1 = np.round(w1 * ss1)
n2 = np.round(w2 * ss1)
n3 = np.round(w3 * ss1)
n4 = np.round(w4 * ss1)
n5 = np.round(w5 * ss1)
n6 = np.round(w6 * ss1)
n7 = np.round(w7 * ss1)
n8 = np.round(w8 * ss1)
n9 = np.round(w9 * ss1)
n10 = np.round(w10 * ss1)
n11 = np.round(w11 * ss1)
#n12 = np.round(w12 * ss2)
#n13 = np.round(w12 * ss2)
#n14 = np.round(w14 * ss2)
#n15 = np.round(w15 * ss2)
#n16 = np.round(w16 * ss2)
#n17 = np.round(w17 * ss2)
#n18 = np.round(w18 * ss2)
#n19 = np.round(w19 * ss2)
#n20 = np.round(w20 * ss2)

print("The sample size for Class 0 is", n0)
print("The sample size for Class 1 is", n1)
print("The sample size for Class 2 is", n2)
print("The sample size for Class 3 is", n3)
print("The sample size for Class 4 is", n4)
print("The sample size for Class 5 is", n5)
print("The sample size for Class 6 is", n6)
print("The sample size for Class 7 is", n7)
print("The sample size for Class 8 is", n8)
print("The sample size for Class 9 is", n9)
print("The sample size for Class 10 is", n10)
print("The sample size for Class 11 is", n11)
#print("The sample size for Class 12 is", n12)
#print("The sample size for Class 13 is", n13)
#print("The sample size for Class 14 is", n14)
#print("The sample size for Class 15 is", n15)
#print("The sample size for Class 16 is", n16)
#print("The sample size for Class 17 is", n17)
#print("The sample size for Class 18 is", n18)
#print("The sample size for Class 19 is", n19)
#print("The sample size for Class 20 is", n20)

The sample size for Class 0 is 42.0
The sample size for Class 1 is 3.0
The sample size for Class 2 is 21.0
The sample size for Class 3 is 0.0
The sample size for Class 4 is 1866.0
The sample size for Class 5 is 16.0
The sample size for Class 6 is 18.0
The sample size for Class 7 is 0.0
The sample size for Class 8 is 304.0
The sample size for Class 9 is 0.0
The sample size for Class 10 is 130.0
The sample size for Class 11 is 3.0


The sample size for Class 0 is 42.0
The sample size for Class 1 is 3.0
The sample size for Class 2 is 21.0
The sample size for Class 3 is 0.0
The sample size for Class 4 is 1866.0
The sample size for Class 5 is 16.0
The sample size for Class 6 is 18.0
The sample size for Class 7 is 0.0
The sample size for Class 8 is 304.0
The sample size for Class 9 is 0.0
The sample size for Class 10 is 130.0
The sample size for Class 11 is 3.0


As we can see, the proportional strategy will leave our rarer classes unsampled, which we don't want. So, as per Olofsson et al 2014, we want to find a happy medium that is between the equal and proportional allocation strategies

Let's use this hybrid strategy, allocating a minimum of ten samples to each class

In [ ]:
min = 30

In [ ]:
remaining_samples = ss2 - (min * 12)

In [ ]:
min * 12

360

360

In [ ]:
remaining_samples

241.0

241.0

In [ ]:
(min * 12) + remaining_samples

601.0

601.0

In [ ]:
wtotal = w0 + w1 + w2 + w3 + w4 + w5 + w6 + w7 + w8 + w9 + w10 + w11

In [ ]:
w0a = w0/wtotal
w1a = w1/wtotal
w2a = w2/wtotal
w3a = w3/wtotal
w4a = w4/wtotal
w5a = w5/wtotal
w6a = w6/wtotal
w7a = w7/wtotal
w8a = w8/wtotal
w9a = w9/wtotal
w10a = w10/wtotal
w11a = w11/wtotal

In [ ]:
N0 = min + np.round(w0a * remaining_samples)
N1 = min + np.round(w1a * remaining_samples)
N2 = min + np.round(w2a * remaining_samples)
N3 = min + np.round(w3a * remaining_samples)
N4 = min + np.round(w4a * remaining_samples)
N5 = min + np.round(w5a * remaining_samples)
N6 = min + np.round(w6a * remaining_samples)
N7 = min + np.round(w7a * remaining_samples)
N8 = min + np.round(w8a * remaining_samples)
N9 = min + np.round(w9a * remaining_samples)
N10 = min + np.round(w10a * remaining_samples)
N11 = min + np.round(w11a * remaining_samples)
N12 = np.float64(0)
N13 = np.float64(0)
N14 = np.float64(0)
N15 = np.float64(0)
N16 = np.float64(0)
N17 = np.float64(0)
N18 = np.float64(0)
N19 = np.float64(0)
N20 = np.float64(0)
N21 = np.float64(0)
N22 = np.float64(0)
N23 = np.float64(0)

#N12 = min + np.round(w12 * remaining_samples)
#N13 = min + np.round(w13 * remaining_samples)
#N14 = min + np.round(w14 * remaining_samples)
#N15 = min + np.round(w15 * remaining_samples)
#N16 = min + np.round(w16 * remaining_samples)
#N17 = min + np.round(w17 * remaining_samples)
#N18 = min + np.round(w18 * remaining_samples)
#N19 = min + np.round(w19 * remaining_samples)
#N20 = min + np.round(w20 * remaining_samples)


In [ ]:
type(N12)

numpy.float64

numpy.float64

In [ ]:
print("The sample size for Class 0 is", N0)
print("The sample size for Class 1 is", N1)
print("The sample size for Class 2 is", N2)
print("The sample size for Class 3 is", N3)
print("The sample size for Class 4 is", N4)
print("The sample size for Class 5 is", N5)
print("The sample size for Class 6 is", N6)
print("The sample size for Class 7 is", N7)
print("The sample size for Class 8 is", N8)
print("The sample size for Class 9 is", N9)
print("The sample size for Class 10 is", N10)
print("The sample size for Class 11 is", N11)
#print("The sample size for Class 12 is", N12)
#print("The sample size for Class 13 is", N13)
#print("The sample size for Class 14 is", N14)
#print("The sample size for Class 15 is", N15)
#print("The sample size for Class 16 is", N16)
#print("The sample size for Class 17 is", N17)
#print("The sample size for Class 18 is", N18)
#print("The sample size for Class 19 is", N19)
#print("The sample size for Class 20 is", N20)

The sample size for Class 0 is 34.0
The sample size for Class 1 is 30.0
The sample size for Class 2 is 32.0
The sample size for Class 3 is 30.0
The sample size for Class 4 is 217.0
The sample size for Class 5 is 32.0
The sample size for Class 6 is 32.0
The sample size for Class 7 is 30.0
The sample size for Class 8 is 61.0
The sample size for Class 9 is 30.0
The sample size for Class 10 is 43.0
The sample size for Class 11 is 30.0


The sample size for Class 0 is 34.0
The sample size for Class 1 is 30.0
The sample size for Class 2 is 32.0
The sample size for Class 3 is 30.0
The sample size for Class 4 is 217.0
The sample size for Class 5 is 32.0
The sample size for Class 6 is 32.0
The sample size for Class 7 is 30.0
The sample size for Class 8 is 61.0
The sample size for Class 9 is 30.0
The sample size for Class 10 is 43.0
The sample size for Class 11 is 30.0


In [ ]:
N0 + N1 + N2 + N3 + N4 + N5 + N6 + N7 + N8 + N9 + N10 + N11

601.0

601.0

# Create feature collection of sampling points

In [ ]:
binary_vp = {
    'min': 0,
    'max': 1,
    'palette': ['000000', 'FFFFFF']
}

In [ ]:
class_values = [ee.Number(0), ee.Number(1), ee.Number(2), ee.Number(3),
                ee.Number(4), ee.Number(5), ee.Number(6), ee.Number(7),
                ee.Number(8), ee.Number(9), ee.Number(10), ee.Number(11)]
                #ee.Number(12), ee.Number(13), ee.Number(14), ee.Number(15),
                #ee.Number(16), ee.Number(17), ee.Number(18), ee.Number(19),
                #ee.Number(20), ee.Number(21), ee.Number(22), ee.Number(23)]

class_points = [ee.Number(N0), ee.Number(N1), ee.Number(N2), ee.Number(N3),
                ee.Number(N4), ee.Number(N5), ee.Number(N6), ee.Number(N7),
                ee.Number(N8), ee.Number(N9), ee.Number(N10), ee.Number(N11)]
                #ee.Number(N12), ee.Number(N13), ee.Number(N14), ee.Number(N15),
                #ee.Number(N16), ee.Number(N17), ee.Number(N18), ee.Number(N19),
                #ee.Number(N20), ee.Number(N21), ee.Number(N22), ee.Number(N23)]

In [ ]:
class_values2 =  ee.List([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
                          #12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

cp2 = [N0, N1, N2, N3, N4, N5, N6, N7, N8, N9, N10, N11]
       #N12, N13, N14, N15, N16, N17, N18, N19, N20, N21, N22, N23]

class_points2 = ee.List(cp2)

In [ ]:
summy = np.sum(cp2)
summy

601.0

601.0

In [ ]:
nump = ee.Number(10)

In [ ]:
'''
my_strat_sample = new_classes21.stratifiedSample(
    region=mpvfc.geometry(),
    numPoints=ee.Number(10),
    classValues=class_values2,
    classPoints=cp2,
    geometries=True,
    scale=10,
    projection=EPSG:4326
)
'''

'\nmy_strat_sample = new_classes21.stratifiedSample(\n    region=mpvfc.geometry(),\n    numPoints=ee.Number(10),\n    classValues=class_values2,\n    classPoints=cp2,\n    geometries=True,\n    scale=10,\n    projection=EPSG:4326\n)\n'

'\nmy_strat_sample = new_classes21.stratifiedSample(\n    region=mpvfc.geometry(),\n    numPoints=ee.Number(10),\n    classValues=class_values2,\n    classPoints=cp2,\n    geometries=True,\n    scale=10,\n    projection=EPSG:4326\n)\n'

In [ ]:
strata.bandNames()

In [ ]:
strat_sample = strata.stratifiedSample(**{
    'region': aoi,
    'numPoints': ee.Number(10),
    'classValues': class_values2,
    'classPoints': cp2,
    'geometries': True,
    'scale': 10,
    'projection': 'EPSG:4326'
})

In [ ]:
'''
strat_sample = new_classes21.stratifiedSample(**{
    'region': rtv,
    'numPoints': ee.Number(10),
    'classValues': class_values2,
    'classPoints': class_points2,
    'geometries': True,
    'scale': 10,
    'projection': 'EPSG:4326'
})
'''

"\nstrat_sample = new_classes21.stratifiedSample(**{\n    'region': rtv,\n    'numPoints': ee.Number(10),\n    'classValues': class_values2,\n    'classPoints': class_points2,\n    'geometries': True,\n    'scale': 10,\n    'projection': 'EPSG:4326'\n})\n"

"\nstrat_sample = new_classes21.stratifiedSample(**{\n    'region': rtv,\n    'numPoints': ee.Number(10),\n    'classValues': class_values2,\n    'classPoints': class_points2,\n    'geometries': True,\n    'scale': 10,\n    'projection': 'EPSG:4326'\n})\n"

In [ ]:
nf_background = strat_sample.filter(ee.Filter.eq('label', 0))
f_background = strat_sample.filter(ee.Filter.eq('label', 1))
nf_trees = strat_sample.filter(ee.Filter.eq('label', 2))
f_trees = strat_sample.filter(ee.Filter.eq('label', 3))
nf_crops = strat_sample.filter(ee.Filter.eq('label', 4))
f_crops = strat_sample.filter(ee.Filter.eq('label', 5))
nf_shrub = strat_sample.filter(ee.Filter.eq('label', 6))
f_shrub = strat_sample.filter(ee.Filter.eq('label', 7))
nf_built = strat_sample.filter(ee.Filter.eq('label', 8))
f_built = strat_sample.filter(ee.Filter.eq('label', 9))
nf_bare = strat_sample.filter(ee.Filter.eq('label', 10))
f_bare = strat_sample.filter(ee.Filter.eq('label', 11))

In [ ]:
strat_sample.getInfo()

{'type': 'FeatureCollection',
 'columns': {'label': 'Short<0, 255>'},
 'properties': {'band_order': ['label']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [73.6936985746228, 30.338847006332806]},
   'id': '0',
   'properties': {'label': 0}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [74.04511951377036, 30.60367035209124]},
   'id': '1',
   'properties': {'label': 0}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [74.31623106651763, 32.62335260537716]},
   'id': '2',
   'properties': {'label': 0}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [74.4115423181627, 32.40236704548376]},
   'id': '3',
   'properties': {'label': 0}},
  {'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [73.74175844232319, 31.219734973940408]},
  

In [ ]:
#s1_binary = new_classes21.eq(1).clip(aoi)
#s1_geom = s1_binary.reduceToVectors(scale = 10, maxPixels = 1e13, geometry = aoi)

#s1_geom_ud = s1_geom.randomPoints(s1_geom, 150)

Map = geemap.Map(center = (lat, lon), zoom = 7)
#Map.addLayer(new_classes21, newvp, 'Strata')
Map.addLayer(strata, strata_vp, 'Strata')
Map.addLayer(strat_sample, {}, 'Stratified Sample')
Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
geemap.ee_export_vector_to_drive(
    strat_sample, description = 'Stratified_Sample_v8', fileFormat='GeoJSON'
)

Exporting Stratified_Sample_v8... Please check the Task Manager from the JavaScript Code Editor.


In [ ]:
n1

3.0

In [ ]:
#new_classes21.bandNames().getInfo()

# References

- Brown, Christopher F., et al. "Dynamic World, Near real-time global 10 m land use land cover mapping." Scientific Data 9.1 (2022): 251.
- Farr, Tom G., and Mike Kobrick. "Shuttle Radar Topography Mission produces a wealth of data." Eos, Transactions American Geophysical Union 81.48 (2000): 583-585.